# Create index base data

This notebook creates the base table from which the index will be calculated
- Selecting area (if needed)
- Filtering grids with nulls
- Feature selection

## Input
- Aligned dataset with all processed features

> Input file for this notebook is produced in `02_dataset_alignment/01_align_index_datasets.ipynb`. The file is provided in the project Google Drive folder.

## Output 
- Aligned dataset with selected features and areas, to be used in index calculation


## Imports and Set-up

In [1]:
%load_ext autoreload
%autoreload 2

# Standard imports
import sys
import pandas as pd

# geospatial
import polars as pl


import re

In [2]:
# Util imports
sys.path.append("../../")  # include parent directory
from src.settings import DATA_DIR
from src.index_utils import add_flag_to_filepath

## Set paths and input paramters

In [3]:
ALIGNED_DIR = DATA_DIR / "aligned"
ALIGNED_DATA_PARQUET = (
    ALIGNED_DIR / "parquets/aligned_index_datasets_consolidated_20240423.parquet"
)

VERSION = pd.to_datetime("today").strftime("%Y%m%d")
INDEX_OUTPUT_DIR = DATA_DIR / "output/component_2"

# if True, drop data with nulls in census data
DROP_NO_CENSUS = True
CHECK_CENSUS_COLS = ["census_population_total_count"]

## Create output filepath based on options


In [4]:
BASE_FPATH = INDEX_OUTPUT_DIR / f"index_base_data.parquet"
OUT_FPATH = BASE_FPATH

if DROP_NO_CENSUS:
    OUT_FPATH = add_flag_to_filepath(OUT_FPATH, "dropnocensus")

# Add version
OUT_FPATH = add_flag_to_filepath(OUT_FPATH, VERSION)

OUT_FPATH

PosixPath('/home/jc_tm/project_repos/immap-evidem-risk-mapping/data/output/component_2/index_base_data_dropnocensus_20240528.parquet')

## Load data

In [5]:
aligned_df = pl.read_parquet(ALIGNED_DATA_PARQUET)
aligned_df

quadkey,MPIO_CCNCT,MPIO_CNMBR,MPIO_CNMBR_EN,DPTO_CNMBR,DPTO_CNMBR_EN,Municipio,Municipio_EN,DPTO_CCDGO,MPIO_CCDGO,MPIO_CRSLC,MPIO_NAREA,MPIO_NANO,SHAPE_AREA,SHAPE_LEN,building_area_m2,building_area_fraction,building_count,census_dwellings_count,census_household_count,census_household_to_dwellings_ratio,census_population_total_count,census_population_density_per_m2,census_population_men_count,census_population_women_count,census_population_0_9_count,census_population_10_19_count,census_population_20_29_count,census_population_30_39_count,census_population_40_49_count,census_population_50_59_count,census_population_60_69_count,census_population_70_79_count,census_population_80_over_count,census_population_dependent_count,census_population_men_percent,census_population_women_percent,…,census_dwellings_internet_unknown_percent,travel_time_to_cities_hr_mean,travel_time_to_cities_hr_median,travel_time_to_cities_hr_max,travel_time_to_cities_hr_min,travel_time_to_cities_hr_count,travel_time_to_cities_hr_sum,ipm_median,ipm_mean,ipm_min,ipm_max,ipm_count,rwi,rwi_flipped,rwi_scaled,rwi_error,poverty_index,travel_time_to_nearest_healthcare_min_mean,travel_time_to_nearest_healthcare_min_median,travel_time_to_nearest_healthcare_min_max,travel_time_to_nearest_healthcare_min_min,travel_time_to_nearest_healthcare_min_count,travel_time_to_nearest_healthcare_min_sum,travel_time_to_nearest_healthcare_walking_only_min_mean,travel_time_to_nearest_healthcare_walking_only_min_median,travel_time_to_nearest_healthcare_walking_only_min_max,travel_time_to_nearest_healthcare_walking_only_min_min,travel_time_to_nearest_healthcare_walking_only_min_count,travel_time_to_nearest_healthcare_walking_only_min_sum,grid_area_m2,cropland_area_m2,cropland_area_fraction,builtup_area_m2,builtup_area_fraction,distance_to_nearest_healthcare_m,travel_time_to_nearest_healthcare_driving_min,travel_time_to_nearest_healthcare_walking_min
str,str,str,str,str,str,str,str,str,str,str,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,str,str,str
"""03223223010033…","""86749""","""SIBUNDOY""","""SIBUNDOY""","""PUTUMAYO""","""PUTUMAYO""","""Sibundoy""","""Sibundoy""","""86""","""749""","""Decreto 1871 d…",97.734625,2020,0.007922,0.511382,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,349.0,349.0,349.0,349.0,1,349.0,null,null,null,null,null,null,null,null,null,null,351.54718,351.54718,351.54718,351.54718,1,351.54718,876.0,876.0,876.0,876.0,1,876.0,23370.477172,0.0,0.0,0.0,0.0,"""13488.58705127…",""">60""",""">60"""
"""03223223010033…","""86749""","""SIBUNDOY""","""SIBUNDOY""","""PUTUMAYO""","""PUTUMAYO""","""Sibundoy""","""Sibundoy""","""86""","""749""","""Decreto 1871 d…",97.734625,2020,0.007922,0.511382,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,380.0,380.0,411.0,349.0,2,760.0,null,null,null,null,null,null,null,null,null,null,351.54718,351.54718,351.54718,351.54718,1,351.54718,876.0,876.0,876.0,876.0,1,876.0,23370.477173,0.0,0.0,0.0,0.0,"""13308.69180533…",""">60""",""">60"""
"""03223223010033…","""86749""","""SIBUNDOY""","""SIBUNDOY""","""PUTUMAYO""","""PUTUMAYO""","""Sibundoy""","""Sibundoy""","""86""","""749""","""Decreto 1871 d…",97.734625,2020,0.007922,0.511382,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,349.0,349.0,349.0,349.0,1,349.0,null,null,null,null,null,null,null,null,null,null,351.54718,351.54718,351.54718,351.54718,1,351.54718,876.0,876.0,876.0,876.0,1,876.0,23370.477172,0.0,0.0,0.0,0.0,"""13321.72933603…",""">60""",""">60"""
"""03223223010033…","""86749""","""SIBUNDOY""","""SIBUNDOY""","""PUTUMAYO""","""PUTUMAYO""","""Sibundoy""","""Sibundoy""","""86""","""749""","""Decreto

In [6]:
aligned_df.schema

OrderedDict([('quadkey', String),
             ('MPIO_CCNCT', String),
             ('MPIO_CNMBR', String),
             ('MPIO_CNMBR_EN', String),
             ('DPTO_CNMBR', String),
             ('DPTO_CNMBR_EN', String),
             ('Municipio', String),
             ('Municipio_EN', String),
             ('DPTO_CCDGO', String),
             ('MPIO_CCDGO', String),
             ('MPIO_CRSLC', String),
             ('MPIO_NAREA', Float64),
             ('MPIO_NANO', Int64),
             ('SHAPE_AREA', Float64),
             ('SHAPE_LEN', Float64),
             ('building_area_m2', Float64),
             ('building_area_fraction', Float64),
             ('building_count', Float64),
             ('census_dwellings_count', Float64),
             ('census_household_count', Float64),
             ('census_household_to_dwellings_ratio', Float64),
             ('census_population_total_count', Float64),
             ('census_population_density_per_m2', Float64),
             ('census_pop

In [7]:
aligned_df.shape

(221152, 123)

In [8]:
process_df = aligned_df.clone()

## Drop null columns

In [9]:
if DROP_NO_CENSUS:
    print(f"Shape before dropping nulls in {CHECK_CENSUS_COLS}: {process_df.shape}")
    process_df = process_df.drop_nulls(CHECK_CENSUS_COLS)
    print(f"Shape after dropping nulls in {CHECK_CENSUS_COLS}: {process_df.shape}")
else:
    print("Nulls in data retained")

Shape before dropping nulls in ['census_population_total_count']: (221152, 123)
Shape after dropping nulls in ['census_population_total_count']: (9910, 123)


## Inspect correlation between features

This part will help us identify features that are highly correlated, which can indicate redundant quantities

In [10]:
# Get the data types of all columns
dtypes = process_df.schema

# Filter out columns with data type String
non_string_columns = [
    col for col, dtype in dtypes.items() if dtype != pl.datatypes.String
]

# Select only non-string columns
features_df = process_df.select(non_string_columns)

# Drop the municipality data
drop_muni_cols = [
    "MPIO_NAREA",
    "MPIO_NANO",
    "SHAPE_AREA",
    "SHAPE_LEN",
]  # List of columns to drop
features_df = features_df.drop(drop_muni_cols)

features_df

building_area_m2,building_area_fraction,building_count,census_dwellings_count,census_household_count,census_household_to_dwellings_ratio,census_population_total_count,census_population_density_per_m2,census_population_men_count,census_population_women_count,census_population_0_9_count,census_population_10_19_count,census_population_20_29_count,census_population_30_39_count,census_population_40_49_count,census_population_50_59_count,census_population_60_69_count,census_population_70_79_count,census_population_80_over_count,census_population_dependent_count,census_population_men_percent,census_population_women_percent,census_population_0_9_percent,census_population_10_19_percent,census_population_20_29_percent,census_population_30_39_percent,census_population_40_49_percent,census_population_50_59_percent,census_population_60_69_percent,census_population_70_79_percent,census_population_80_over_percent,census_population_dependent_percent,census_population_edu_level_primary_count,census_population_edu_level_secondary_count,census_population_edu_level_technical_professional_count,census_population_edu_level_postgraduate_count,census_population_edu_level_tertiary_count,…,census_dwellings_internet_unknown_count,census_dwellings_internet_service_percent,census_dwellings_no_internet_service_percent,census_dwellings_internet_unknown_percent,travel_time_to_cities_hr_mean,travel_time_to_cities_hr_median,travel_time_to_cities_hr_max,travel_time_to_cities_hr_min,travel_time_to_cities_hr_count,travel_time_to_cities_hr_sum,ipm_median,ipm_mean,ipm_min,ipm_max,ipm_count,rwi,rwi_flipped,rwi_scaled,rwi_error,poverty_index,travel_time_to_nearest_healthcare_min_mean,travel_time_to_nearest_healthcare_min_median,travel_time_to_nearest_healthcare_min_max,travel_time_to_nearest_healthcare_min_min,travel_time_to_nearest_healthcare_min_count,travel_time_to_nearest_healthcare_min_sum,travel_time_to_nearest_healthcare_walking_only_min_mean,travel_time_to_nearest_healthcare_walking_only_min_median,travel_time_to_nearest_healthcare_walking_only_min_max,travel_time_to_nearest_healthcare_walking_only_min_min,travel_time_to_nearest_healthcare_walking_only_min_count,travel_time_to_nearest_healthcare_walking_only_min_sum,grid_area_m2,cropland_area_m2,cropland_area_fraction,builtup_area_m2,builtup_area_fraction
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64
0.0,0.0,0.0,0.0,0.0,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,…,0.0,null,null,null,338.0,338.0,338.0,338.0,1,338.0,null,null,null,null,null,null,null,null,null,null,323.265839,323.265839,323.265839,323.265839,1,323.265839,903.0,903.0,903.0,903.0,1,903.0,23370.477172,7632.849648,0.326602,0.0,0.0
0.0,0.0,0.0,0.0,0.0,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,…,0.0,null,null,null,375.5,375.5,388.0,363.0,2,751.0,null,null,null,null,null,null,null,null,null,null,323.265839,323.265839,323.265839,323.265839,1,323.265839,903.0,903.0,903.0,903.0,1,903.0,23370.477173,14137.244083,0.604919,0.0,0.0
0.0,0.0,0.0,0.0,0.0,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,…,0.0,null,null,null,375.5,375.5,388.0,363.0,2,751.0,null,null,null,null,null,null,null,null,null,null,323.265839,323.265839,323.265839,323.265839,1,323.265839,903.0,903.0,903.0,903.0,1,903.0,23370.477172,5867.936041,0.251083,0.0,0.0
150.580882,0.006443,1.0,2.0,2.0,1.0,3.5,0.00015,1.5,2.0,1.0,0.0,1.0,1.0,0.0,0.5,0.0,0.0,0.0,1.0,0.428571,0.571429,0.285714,0.0,0.285714,0.285714,0.0,0.142857,0.0,0.0,0.0,0.285714,1.0,0.5,1.5,0.0,1.5,…,0

In [11]:
## Get correlation table
## TODO: add as util
long = features_df.select(
    [pl.corr(pl.all(), pl.col(c)).name.suffix("|" + c) for c in features_df.columns]
).melt()
table = long.select(
    pl.col("variable").map_elements(lambda x: x.split("|")[0]).alias("var1"),
    pl.col("variable").map_elements(lambda x: x.split("|")[1]).alias("var2"),
    pl.col("value").alias("corr"),
).sort(by="corr")

with pl.Config(fmt_str_lengths=2000):
    display(table)
    print(table.glimpse())

var1,var2,corr
str,str,f64
"""census_population_men_percent""","""census_population_women_percent""",-1.0
"""census_population_women_percent""","""census_population_men_percent""",-1.0
"""rwi_scaled""","""rwi""",-1.0
"""rwi""","""rwi_scaled""",-1.0
"""rwi_flipped""","""rwi""",-1.0
…,…,…
"""builtup_area_fraction""","""grid_area_m2""",NaN
"""grid_area_m2""","""cropland_area_m2""",NaN
"""grid_area_m2""","""cropland_area_fraction""",NaN


Rows: 11025
Columns: 3
$ var1 <str> 'census_population_men_percent', 'census_population_women_percent', 'rwi_scaled', 'rwi', 'rwi_flipped', 'rwi', 'census_dwellings_no_garbage_collection_service_percent', 'census_dwellings_garbage_collection_service_percent', 'census_dwellings_no_water_service_percent', 'census_dwellings_water_service_percent'
$ var2 <str> 'census_population_women_percent', 'census_population_men_percent', 'rwi', 'rwi_scaled', 'rwi', 'rwi_flipped', 'census_dwellings_garbage_collection_service_percent', 'census_dwellings_no_garbage_collection_service_percent', 'census_dwellings_water_service_percent', 'census_dwellings_no_water_service_percent'
$ corr <f64> -1.0000000000000147, -1.000000000000014, -1.0000000000000013, -1.0000000000000013, -1.0, -1.0, -0.8801486141137306, -0.8801486141137306, -0.8681018862309762, -0.8681018862309762

None


## Select features
- Get id cols (quadkey, MPIO_CNMBR_EN, MPIO_CCNCT)
- Get the median values for raster zonal stats
- Get percent incidence for census data
- Get area fraction columns for buildings and landcover
- Get specific columns from census data: population density, total population, and total number of dwellings and households
- Drop population_density and population_density UNadj (from WorldPop)

In [12]:
id_cols = ["quadkey", "MPIO_CNMBR_EN", "MPIO_CCNCT"]
manual_cols = [
    "census_dwellings_count",
    "census_household_count",
    "census_population_total_count",
    "census_population_density_per_m2",
    "census_household_to_dwellings_ratio",
    "poverty_index",
    "rwi_scaled",
    "distance_to_nearest_healthcare_m",
    "travel_time_to_nearest_healthcare_driving_min",
    "travel_time_to_nearest_healthcare_walking_min",
]

# drop_cols = ["population_density_UNadj_median", "population_density_median"]
drop_cols = []

In [13]:
median_pattern = re.compile(".*_median$")
median_cols = [col for col in process_df.columns if median_pattern.match(col)]
median_cols

['travel_time_to_cities_hr_median',
 'ipm_median',
 'travel_time_to_nearest_healthcare_min_median',
 'travel_time_to_nearest_healthcare_walking_only_min_median']

In [14]:
census_percent_pattern = re.compile("^census_[^_]+(?:_[^_]+)*_percent$")
census_percent_cols = [
    col for col in process_df.columns if census_percent_pattern.match(col)
]
census_percent_cols

['census_population_men_percent',
 'census_population_women_percent',
 'census_population_0_9_percent',
 'census_population_10_19_percent',
 'census_population_20_29_percent',
 'census_population_30_39_percent',
 'census_population_40_49_percent',
 'census_population_50_59_percent',
 'census_population_60_69_percent',
 'census_population_70_79_percent',
 'census_population_80_over_percent',
 'census_population_dependent_percent',
 'census_population_edu_level_primary_percent',
 'census_population_edu_level_secondary_percent',
 'census_population_edu_level_technical_professional_percent',
 'census_population_edu_level_postgraduate_percent',
 'census_population_edu_level_tertiary_percent',
 'census_population_edu_level_none_percent',
 'census_population_edu_level_unknown_percent',
 'census_dwellings_water_service_percent',
 'census_dwellings_no_water_service_percent',
 'census_dwellings_sewerage_service_percent',
 'census_dwellings_no_sewerage_service_percent',
 'census_dwellings_garbage

In [15]:
area_fraction_pattern = re.compile(".*_area_fraction$")
area_fraction_cols = [
    col for col in process_df.columns if area_fraction_pattern.match(col)
]
area_fraction_cols

['building_area_fraction', 'cropland_area_fraction', 'builtup_area_fraction']

In [16]:
all_features = manual_cols
all_features += median_cols + census_percent_cols + area_fraction_cols
all_features = [x for x in all_features if x not in drop_cols]
all_features = list(dict.fromkeys(all_features))  # dedup while keeping order
all_features

['census_dwellings_count',
 'census_household_count',
 'census_population_total_count',
 'census_population_density_per_m2',
 'census_household_to_dwellings_ratio',
 'poverty_index',
 'rwi_scaled',
 'distance_to_nearest_healthcare_m',
 'travel_time_to_nearest_healthcare_driving_min',
 'travel_time_to_nearest_healthcare_walking_min',
 'travel_time_to_cities_hr_median',
 'ipm_median',
 'travel_time_to_nearest_healthcare_min_median',
 'travel_time_to_nearest_healthcare_walking_only_min_median',
 'census_population_men_percent',
 'census_population_women_percent',
 'census_population_0_9_percent',
 'census_population_10_19_percent',
 'census_population_20_29_percent',
 'census_population_30_39_percent',
 'census_population_40_49_percent',
 'census_population_50_59_percent',
 'census_population_60_69_percent',
 'census_population_70_79_percent',
 'census_population_80_over_percent',
 'census_population_dependent_percent',
 'census_population_edu_level_primary_percent',
 'census_population_e

In [17]:
selected_features_df = process_df.select(
    [col for col in process_df.columns if col in id_cols + all_features]
)
selected_features_df

quadkey,MPIO_CCNCT,MPIO_CNMBR_EN,building_area_fraction,census_dwellings_count,census_household_count,census_household_to_dwellings_ratio,census_population_total_count,census_population_density_per_m2,census_population_men_percent,census_population_women_percent,census_population_0_9_percent,census_population_10_19_percent,census_population_20_29_percent,census_population_30_39_percent,census_population_40_49_percent,census_population_50_59_percent,census_population_60_69_percent,census_population_70_79_percent,census_population_80_over_percent,census_population_dependent_percent,census_population_edu_level_primary_percent,census_population_edu_level_secondary_percent,census_population_edu_level_technical_professional_percent,census_population_edu_level_postgraduate_percent,census_population_edu_level_tertiary_percent,census_population_edu_level_none_percent,census_population_edu_level_unknown_percent,census_dwellings_water_service_percent,census_dwellings_no_water_service_percent,census_dwellings_sewerage_service_percent,census_dwellings_no_sewerage_service_percent,census_dwellings_garbage_collection_service_percent,census_dwellings_no_garbage_collection_service_percent,census_dwellings_w_elec_percent,census_dwellings_wo_elec_percent,census_dwellings_ind_percent,census_dwellings_eth_percent,census_dwellings_internet_service_percent,census_dwellings_no_internet_service_percent,census_dwellings_internet_unknown_percent,travel_time_to_cities_hr_median,ipm_median,rwi_scaled,poverty_index,travel_time_to_nearest_healthcare_min_median,travel_time_to_nearest_healthcare_walking_only_min_median,cropland_area_fraction,builtup_area_fraction,distance_to_nearest_healthcare_m,travel_time_to_nearest_healthcare_driving_min,travel_time_to_nearest_healthcare_walking_min
str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str
"""03223223010211…","""86749""","""SIBUNDOY""",0.0,0.0,0.0,null,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,338.0,null,null,null,323.265839,903.0,0.326602,0.0,"""12120.58186516…",""">60""",""">60"""
"""03223223010211…","""86749""","""SIBUNDOY""",0.0,0.0,0.0,null,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,375.5,null,null,null,323.265839,903.0,0.604919,0.0,"""11609.33756971…",""">60""",""">60"""
"""03223223010211…","""86749""","""SIBUNDOY""",0.0,0.0,0.0,null,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,375.5,null,null,null,323.265839,903.0,0.251083,0.0,"""11620.25954237…",""">60""",""">60"""
"""03223223010230…","""86749""","""SIBUNDOY""",0.006443,2.0,2.0,1.0,3.5,0.00015,0.428571,0.571429,0.285714,0.0,0.285714,0.285714,0.0,0.142857,0.0,0.0,0.0,0.285714,0.285714,0.142857,0.428571,0.0,0.428571,0.0,0.142857,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,311.0,null,74.526678,74.526678,319.955475,738.0,0.0,0.0,"""6756.959846086…",""">60""",""">60"""
"""03223223010232…","""86749""","""SIBUNDOY""",0.002973,2.0,2.0,1.0,3.5,0.00015,0.428571,0.571429,0.285714,0.0,0.285714,0.285714,0.0,0.142857,0.0,0.0,0.0,0.285714,0.285714,0.142857,0.428571,0.0,0.428571,0.0,0.142857,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,206.0,null,74.526678,74.526678,138.096603,556.0,0.0,0.0,"""5877.726189470…",""">60""",""">60"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""03223102222032…","""54250""","""EL TARRA""",0.00261,0.671875,0.5,0.744186,1.8828125,0.000081,0.551867,0.448133,0.205394,0.246888,0.188797,0.122407,0.089212,0.082988,0.037344,0.018672,0.008299,0.26971,0.560

In [18]:
selected_features_df.schema

OrderedDict([('quadkey', String),
             ('MPIO_CCNCT', String),
             ('MPIO_CNMBR_EN', String),
             ('building_area_fraction', Float64),
             ('census_dwellings_count', Float64),
             ('census_household_count', Float64),
             ('census_household_to_dwellings_ratio', Float64),
             ('census_population_total_count', Float64),
             ('census_population_density_per_m2', Float64),
             ('census_population_men_percent', Float64),
             ('census_population_women_percent', Float64),
             ('census_population_0_9_percent', Float64),
             ('census_population_10_19_percent', Float64),
             ('census_population_20_29_percent', Float64),
             ('census_population_30_39_percent', Float64),
             ('census_population_40_49_percent', Float64),
             ('census_population_50_59_percent', Float64),
             ('census_population_60_69_percent', Float64),
             ('census_population

In [19]:
selected_features_df.shape

(9910, 52)

In [20]:
# check for nulls in final output
nulls_check_df = selected_features_df.select(pl.all().is_null().sum())
nulls_check_df

quadkey,MPIO_CCNCT,MPIO_CNMBR_EN,building_area_fraction,census_dwellings_count,census_household_count,census_household_to_dwellings_ratio,census_population_total_count,census_population_density_per_m2,census_population_men_percent,census_population_women_percent,census_population_0_9_percent,census_population_10_19_percent,census_population_20_29_percent,census_population_30_39_percent,census_population_40_49_percent,census_population_50_59_percent,census_population_60_69_percent,census_population_70_79_percent,census_population_80_over_percent,census_population_dependent_percent,census_population_edu_level_primary_percent,census_population_edu_level_secondary_percent,census_population_edu_level_technical_professional_percent,census_population_edu_level_postgraduate_percent,census_population_edu_level_tertiary_percent,census_population_edu_level_none_percent,census_population_edu_level_unknown_percent,census_dwellings_water_service_percent,census_dwellings_no_water_service_percent,census_dwellings_sewerage_service_percent,census_dwellings_no_sewerage_service_percent,census_dwellings_garbage_collection_service_percent,census_dwellings_no_garbage_collection_service_percent,census_dwellings_w_elec_percent,census_dwellings_wo_elec_percent,census_dwellings_ind_percent,census_dwellings_eth_percent,census_dwellings_internet_service_percent,census_dwellings_no_internet_service_percent,census_dwellings_internet_unknown_percent,travel_time_to_cities_hr_median,ipm_median,rwi_scaled,poverty_index,travel_time_to_nearest_healthcare_min_median,travel_time_to_nearest_healthcare_walking_only_min_median,cropland_area_fraction,builtup_area_fraction,distance_to_nearest_healthcare_m,travel_time_to_nearest_healthcare_driving_min,travel_time_to_nearest_healthcare_walking_min
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,1014,0,0,1032,1032,1032,1032,1032,1032,1032,1032,1032,1032,1032,1032,1032,1032,1032,1032,1032,1032,1032,1014,1014,1014,1014,1014,1014,1014,1014,1014,1014,1014,1014,1014,4,8105,1000,926,0,0,0,0,0,0,0


## Output table

In [21]:
selected_features_df.write_parquet(OUT_FPATH)